### 개요
- 주택가격을 예측하는 데 필요한 Kaggle 데이터를 불러와서 빅쿼리에 저장하는 실습 진행
- 데이터를 불러와서 LightGBM를 활용하여 머신러닝을 만든다.
### I. 사전 준비작업
- Kaggle API 설치 및 연동해서 GCP에 데이터를 적재하는 것까지 진행한다.
### (1) Kaggle API 설치
- 구글 코랩에서 API를 불러오려면 다음 소스코드를 실행한다.

In [1]:
!pip install kaggle

### (2) Kaggle Token 다운로드
- Kaggle에서 API Token을 다운로드 받는다.
- [Kaggle]-[My Account]-[API]-[Create New API Token]을 누르면 kaggle.json 파일이 다운로드 된다.
- 이 파일을 바탕화면에 옮긴 뒤, 아래 코드를 실행 시킨다.

In [2]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# kaggle.json을 아래 폴더로 옮긴 뒤, file을 사용할 수 있도록 권한을 부여한다. 
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
uploaded file "kaggle.json" with length 64 bytes


- 실제 kaggle.json 파일이 업로드 되었다는 뜻이다.

In [3]:
ls -1ha ~/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


In [4]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started      Kudos        191           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2948           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      22247           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       5052            True  
connectx                                       2030-01-01 00:00:00  Getting Started  Knowledge        768           False  
nlp-getting-started                            2030-01-01 00:00:00  Getting Started      Kudos       1511           False  
competit

In [6]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

data_description.txt: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
!ls

adc.json	      results.csv  sample_submission.csv  train.csv
data_description.txt  sample_data  test.csv


In [58]:
import pandas as pd
from pandas.io import gbq

# import sample_submission file
sample_submission = pd.read_csv('sample_submission.csv')

# Connect to Google Cloud API and Upload DataFrame
sample_submission.to_gbq(destination_table='house_price.sample_submission', 
                  project_id='myproject-283503', 
                  if_exists='replace')

1it [00:02,  2.94s/it]


In [59]:
# import train file 
train = pd.read_csv('train.csv')

In [60]:
print(train.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
#숫자가 있으면 못들어가니, 1st앞에 my1st로 다 바꾼다.

In [61]:
colnames_dict = {"1stFlrSF": "my1stFlrSF", "2ndFlrSF": "my2ndFlrSF", "3SsnPorch": "my3SsnPorch"}

In [62]:
# Connect to Google Cloud API and Upload DataFrame
train = train.rename(columns=colnames_dict)
train.to_gbq(destination_table='house_price.train', 
                  project_id='myproject-283503', 
                  if_exists='replace')

1it [00:05,  5.83s/it]


In [63]:
# Connect to Google Cloud API and Upload DataFrame
test = pd.read_csv('test.csv')
test = test.rename(columns=colnames_dict)
test.to_gbq(destination_table='house_price.test', 
            project_id='myproject-283503', 
            if_exists='replace')

1it [00:03,  3.38s/it]


In [64]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict

## (2) 데이터 불러오기

In [65]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


- 먼저 훈련 데이터를 불러온다.

In [66]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'myproject-283503'
client = bigquery.Client(project=project_id)

df_train = client.query('''
  SELECT 
      * 
  FROM `myproject-283503.house_price.train`
  ''').to_dataframe()

df_train.shape

(1460, 81)

  - 그 다음은 테스트 데이터를 불러온다.

In [67]:
df_test = client.query('''
  SELECT 
      * 
  FROM `myproject-283503.house_price.test`
  ''').to_dataframe()

df_test.shape

(1459, 80)

- 아래 코드는 출력 시, 전체 Column에 대해 확인할 수 있음

In [68]:
pd.options.display.max_columns = None 
# df_train.describe()

### (3) 결측 데이터 확인

In [69]:
# data set의 Percent 구하는 함수를 짜보자. 
def check_fill_na(data):
  new_df = data.copy()
  new_df_na = (new_df.isnull().sum() / len(new_df)) * 100
  new_df_na.sort_values(ascending=False).reset_index(drop=True)
  new_df_na = new_df_na.drop(new_df_na[new_df_na == 0].index).sort_values(ascending=False)
  return new_df_na

check_fill_na(df_train)

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
FireplaceQu     47.260274
LotFrontage     17.739726
GarageYrBlt      5.547945
GarageType       5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
BsmtQual         2.534247
MasVnrArea       0.547945
MasVnrType       0.547945
Electrical       0.068493
dtype: float64

### (4) 주요 함수 정의
  - 수치형과 범주형 데이터 결측치의 보간에 관한 함수를 정의한다.

In [70]:
def fill_missing(df, cols, val):
    """ val 입력값을 넣는다. """
    for col in cols:
        df[col] = df[col].fillna(val)

def fill_missing_with_mode(df, cols):
    """ 최대 빈도수를 넣는다. """
    for col in cols:
        df[col] = df[col].fillna(df[col].mode()[0])
        
def addlogs(res, cols):
    """ 로그 변환 """
    m = res.shape[1]
    for c in cols:
        res = res.assign(newcol=pd.Series(np.log(1.01+res[c])).values)   
        res.columns.values[m] = c + '_log'
        m += 1
    return res

- 1층, 2층, 3층의 면적을 합친 전체 total을 구해본다.
### (5) 전체 면적 데이터 추가
- 가정의 전체 면적을 더해서 추가 변수를 만든다.

In [71]:
df_train['TotalSF'] = df_train['TotalBsmtSF'] + df_train['my1stFlrSF'] + df_train['my2ndFlrSF']
df_train['TotalSF']

0       2765
1       1521
2       1868
3       2856
4       1632
        ... 
1455    1683
1456    2871
1457    3285
1458    2078
1459    1920
Name: TotalSF, Length: 1460, dtype: int64

- 전체 수치형 데이터에 log transformation을 해준다.

In [72]:
loglist = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
            'TotalBsmtSF','my1stFlrSF','my2ndFlrSF','LowQualFinSF','GrLivArea',
            'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
            'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF',
            'EnclosedPorch','my3SsnPorch','ScreenPorch','PoolArea','MiscVal','YearRemodAdd','TotalSF']

df_train = addlogs(df_train, loglist)

## (6) 타겟변수 로그변환
- 데이터가 작기 때문에, 모형의 안정성을 위해 로그변환을 해준다.

In [73]:
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])

### (7) 결측치 데이터 보간
- 결측치 데이터를 보간한다.

In [74]:
# 우선, 결측치가 있는 것 중, 범주형 데이터는 "None"으로 확인
fill_missing(df_train, ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", 
                        "GarageType", "GarageFinish", "GarageQual", "GarageCond",
                       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       "MasVnrType", "MSSubClass"], "None") 

# 수치형 데이터는 0으로 보간
fill_missing(df_train, ["GarageYrBlt", "GarageArea", "GarageCars",
                       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                       "MasVnrArea"], 0)
# 그 중, 일부는 빈도수로 채워 넣는다.  
fill_missing_with_mode(df_train, ["MSZoning", "KitchenQual", "Exterior1st", "Exterior2nd", "SaleType"])
fill_missing(df_train, ["Functional"],"Typ")

### (8) 변수 삭제
- 1개의 값만 존재하는 데이터는 삭제한다.

In [75]:
df_train.drop(['Utilities'], axis=1, inplace=True)

### (9) 이상치 제거
- 적은 데이터에서 상위 또는 하위 이상치가 발생하는 것은 좋지 않다. 따라서, 해당 관측치는 제거한다.

In [76]:
df_train.drop(df_train[(df_train['OverallQual']<5) & (df_train['SalePrice']>200000)].index, inplace=True)
df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index, inplace=True)
df_train.reset_index(drop=True, inplace=True)

### (10) 재범주화
- 몇몇 수치형 데이터는 사실 범주형 데이터에 가깝다.
- 따라서, 이를 문자형으로 바꾼다.

In [77]:
df_train['MSSubClass'] = df_train['MSSubClass'].apply(str)
df_train['YrSold'] = df_train['YrSold'].astype(str)
df_train['MoSold'] = df_train['MoSold'].astype(str)

### (11) 범주형 데이터 다루기
- 이제 범주형 데이터를 원핫 인코딩으로 변환한다.
- 원핫 인코딩으로 변환하는 이유는, 알고리즘은 수치형으로 되어 있기 때문에 그렇다.

In [78]:
def fix_missing_cols(in_train, in_test):
    missing_cols = set(in_train.columns) - set(in_test.columns)
    # 테스트 데이터와 훈련 데이터의 컬럼을 동일하게 하는 코드는 작성한다. 
    for c in missing_cols:
        in_test[c] = 0
    # 순서를 동일하게 만든다. 
    in_test = in_test[in_train.columns]
    return in_test

def dummy_encode(in_df_train, in_df_test):
    df_train = in_df_train
    df_test = in_df_test
    categorical_feats = [
        f for f in df_train.columns if df_train[f].dtype == 'object'
    ]
    print(categorical_feats)
    for f_ in categorical_feats:
        prefix = f_
        df_train = pd.concat([df_train, pd.get_dummies(df_train[f_], prefix=prefix)], axis=1).drop(f_, axis=1)
        df_test = pd.concat([df_test, pd.get_dummies(df_test[f_], prefix=prefix)], axis=1).drop(f_, axis=1)
        df_test = fix_missing_cols(df_train, df_test)
    return df_train, df_test

- 훈련 데이터와 테스트 데이터의 크기가 다르면 예측 시, 에러가 발생한다.

In [79]:
df_train, df_test = dummy_encode(df_train, df_test)
print("Shape train: %s, test: %s" % (df_train.shape, df_test.shape))

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']
Shape train: (1456, 361), test: (1459, 361)


### III. 머신러닝 모형 개발
- 이제 LightGBM을 활용하여 머신러닝 모형을 개발한다.
#### (1) 종속변수 처리
- 종속변수를 y 객체로 저장한다.

In [80]:
y = df_train["SalePrice"]
y.sample(3)

1262    12.095147
869     12.031725
525     12.843974
Name: SalePrice, dtype: float64

- 훈련 및 테스트 데이터의 변수를 삭제한다.

In [81]:
df_train.drop(["SalePrice"], axis=1, inplace=True)
df_test.drop(["SalePrice"], axis=1, inplace=True)

print("Shape train: %s, test: %s" % (df_train.shape, df_test.shape))

Shape train: (1456, 360), test: (1459, 360)


### (2) 데이터셋 분리
- 데이터셋을 분리한다.

In [82]:
X_train, X_test, y_train, y_test = train_test_split( df_train, y, test_size=0.2, random_state=42)

### (3) LightGBM 파라미터 정의
- LightGBM 파라미터 정의는 다음 메뉴얼을 읽고 적용한다.
- LightGBM 파라미터 메뉴얼

https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [83]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

### (4) 모델 정의
- 이제 모델을 정의한다.

In [84]:
gbm = lgb.LGBMRegressor(**hyper_params)

### (5) 모델 학습
- 이제 모델을 학습한다.

In [85]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=1000)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 1	valid_0's l1: 0.333038	valid_0's l2: 0.186477
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's auc: 1	valid_0's l1: 0.331606	valid_0's l2: 0.185073
[3]	valid_0's auc: 1	valid_0's l1: 0.330192	valid_0's l2: 0.183684
[4]	valid_0's auc: 1	valid_0's l1: 0.328797	valid_0's l2: 0.18231
[5]	valid_0's auc: 1	valid_0's l1: 0.327404	valid_0's l2: 0.18095
[6]	valid_0's auc: 1	valid_0's l1: 0.326023	valid_0's l2: 0.179603
[7]	valid_0's auc: 1	valid_0's l1: 0.324652	valid_0's l2: 0.17828
[8]	valid_0's auc: 1	valid_0's l1: 0.323279	valid_0's l2: 0.176947
[9]	valid_0's auc: 1	valid_0's l1: 0.321952	valid_0's l2: 0.175652
[10]	valid_0's auc: 1	valid_0's l1: 0.320633	valid_0's l2: 0.174352
[11]	valid_0's auc: 1	valid_0's l1: 0.319239	valid_0's l2: 0.172985
[12]	valid_0's auc: 1	valid_0's l1: 0.317859	valid_0's l2: 0.171633
[13]	valid_0's auc: 1	valid_0's l1: 0.316502	valid_0's l2: 0.170309
[14]	valid_0's auc: 1	valid_0's l1: 0.315116	valid_0's l2: 0.168

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.005, max_bin=512,
              max_depth=8, metric=['l2', 'auc'], min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_iterations=100000, num_leaves=128,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=0)

### (6) 모델 평가
- 모델을 평가한다. (RMSE)

In [86]:
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))

The rmse of prediction is: 0.02951


### (7) 결과 제출
- 이제 결과를 제출한다.

In [87]:
test_pred = np.expm1(gbm.predict(df_test, num_iteration=gbm.best_iteration_))
df_test["SalePrice"] = test_pred
df_test.to_csv("results.csv", columns=["Id", "SalePrice"], index=False)
